In [1]:
!pip install gensim --user
!pip install textblob

In [40]:
import pandas as pd
import numpy as np
import re

#Timing
from tqdm import tqdm

#Word Embeddings
import nltk
from textblob import TextBlob
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## EDA

In [41]:
scored_tweets = pd.read_csv('scored_tweets.csv')

In [39]:
# scored_tweets

In [42]:
scored_tweets.columns

Index(['created_at', 'text', 'username', 'screen_name', 'verified',
       'followers_count', 'TOXICITY', 'INSULT', 'PROFANITY', 'THREAT',
       'SEXUALLY_EXPLICIT'],
      dtype='object')

In [43]:
scored_tweets.corr()

,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
TOXICITY,1.000000,0.945739,0.921274,0.572428,0.700610
INSULT,0.945739,1.000000,0.827742,0.464412,0.542923
PROFANITY,0.921274,0.827742,1.000000,0.442225,0.758937
THREAT,0.572428,0.464412,0.442225,1.000000,0.425812
SEXUALLY_EXPLICIT,0.700610,0.542923,0.758937,0.425812,1.000000


In [44]:
scored_tweets.head(3)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,created_at,text,username,screen_name,verified,followers_count,0.042518,0.015740,0.037332,0.043029,0.040426
1,Thu Dec 02 15:39:21 +0000 2021,This needs to become a real thing. I wanna see...,GhostFedoraTWEET,GhostyRBLX,False,28,0.552407,0.666591,0.313011,0.431656,0.100750
2,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members love u more jake,aii.laa,jayenaii,False,85,0.084299,0.061555,0.109649,0.069504,0.119767


## New Preprocessing

In [45]:
scored_tweets.dropna(subset=['text'], inplace=True)

In [46]:
from nltk.tokenize import sent_tokenize, word_tokenize

#tokenizes every tweet
tokenizer = nltk.RegexpTokenizer(r"\w+")
scored_tweets['processed'] = scored_tweets['text'].apply(tokenizer.tokenize)
scored_tweets['processed']

0                                                  [text]
1       [This, needs, to, become, a, real, thing, I, w...
2                  [ENHYPEN_members, love, u, more, jake]
3       [ENHYPEN_members, i, hope, you, ll, get, a, go...
4       [Mal_DuBois, It, was, dragged, across, on, a, ...
                              ...                        
1957    [Nurseries, in, Glasgow, Come, along, to, a, t...
1958    [jimin, is, a, real, life, angel, https, t, co...
1959    [8Ball, Who, Want, it, 600, 00, If, You, WIN, ...
1960               [I, no, longer, have, time, to, waste]
1961    [BugCatNoms, Awwwwww, Puppet, Bugalo, Cat, Tha...
Name: processed, Length: 1962, dtype: object

In [47]:
from nltk.corpus import stopwords 

#Widely used stopword library
stop_words = set(stopwords.words('english'))

#gets rid of numbers
stop_words.update([str(x) for x in np.arange(10)])

#apply to tokens
scored_tweets['processed'] = scored_tweets.processed.apply(lambda x: [i for i in x if not i in stop_words])
scored_tweets['processed']

0                                                  [text]
1       [This, needs, become, real, thing, I, wanna, s...
2                        [ENHYPEN_members, love, u, jake]
3       [ENHYPEN_members, hope, get, good, rest, fight...
4               [Mal_DuBois, It, dragged, across, dinghy]
                              ...                        
1957    [Nurseries, Glasgow, Come, along, taster, sess...
1958    [jimin, real, life, angel, https, co, 1LsoCrsG1I]
1959    [8Ball, Who, Want, 600, 00, If, You, WIN, If, ...
1960                             [I, longer, time, waste]
1961    [BugCatNoms, Awwwwww, Puppet, Bugalo, Cat, Tha...
Name: processed, Length: 1962, dtype: object

In [48]:
scored_tweets['processed'] = [' '.join(l) for l in scored_tweets['processed']]
scored_tweets['processed']

0                                                    text
1       This needs become real thing I wanna see full ...
2                             ENHYPEN_members love u jake
3       ENHYPEN_members hope get good rest fighting to...
4                     Mal_DuBois It dragged across dinghy
                              ...                        
1957    Nurseries Glasgow Come along taster session fi...
1958            jimin real life angel https co 1LsoCrsG1I
1959    8Ball Who Want 600 00 If You WIN If I Win TRYI...
1960                                  I longer time waste
1961    BugCatNoms Awwwwww Puppet Bugalo Cat Thank u n...
Name: processed, Length: 1962, dtype: object

In [49]:
#Convert Scores to Round Numbers
scored_tweets['TOXICITY'] = round(scored_tweets['TOXICITY'] * 10, 0)
scored_tweets['PROFANITY'] = round(scored_tweets['PROFANITY'] * 10, 0)
scored_tweets['SEXUALLY_EXPLICIT'] = round(scored_tweets['SEXUALLY_EXPLICIT'] * 10, 0)

In [50]:
scored_tweets.head(5)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT,processed
0,created_at,text,username,screen_name,verified,followers_count,0.0,0.015740,0.0,0.043029,0.0,text
1,Thu Dec 02 15:39:21 +0000 2021,This needs to become a real thing. I wanna see...,GhostFedoraTWEET,GhostyRBLX,False,28,6.0,0.666591,3.0,0.431656,1.0,This needs become real thing I wanna see full ...
2,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members love u more jake,aii.laa,jayenaii,False,85,1.0,0.061555,1.0,0.069504,1.0,ENHYPEN_members love u jake
3,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members i hope you'll get a good rest...,ً,igpshluv,False,2337,1.0,0.103891,1.0,0.506230,1.0,ENHYPEN_members hope get good rest fighting to...
4,Thu Dec 02 15:39:21 +0000 2021,@Mal_DuBois It was dragged across on a dinghy...,Gavlar87,gavlar87,False,149,2.0,0.123806,1.0,0.298187,2.0,Mal_DuBois It dragged across dinghy


In [51]:
#Confirms range of values for labels
scored_tweets['TOXICITY'].value_counts()

1.0     993
2.0     370
3.0     128
0.0     126
4.0      82
9.0      69
5.0      63
7.0      49
6.0      46
8.0      31
10.0      5
Name: TOXICITY, dtype: int64

## Models

In [60]:
import pickle

In [33]:
!pip install --upgrade xgboost

  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.5.0
    Uninstalling xgboost-1.5.0:
      Successfully uninstalled xgboost-1.5.0


In [52]:
def split_data(df=scored_tweets, feature='processed', label='TOXICITY'):
    from sklearn.model_selection import train_test_split
    
    train_data, test_data = train_test_split(df, test_size=0.25, random_state=7)
    X_train, X_test = np.asarray(train_data['processed']), np.asarray(test_data['processed'])
    y_train, y_test = np.asarray(train_data[label]), np.asarray(test_data[label])
    
    return X_train, X_test, y_train, y_test

In [53]:
def TFIDF(X_train, X_test):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    train_tfidf = vectorizer.fit_transform(X_train)
    test_tfidf = vectorizer.transform(X_test)
    
    return train_tfidf, test_tfidf    

In [58]:
def XGBoost_Classifier(feature_category):
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    model = XGBClassifier()
    model.fit(train_tfidf, y_train)
    
    y_pred = model.predict(test_tfidf)
    acc = accuracy_score(y_pred, y_test)
    
    return (acc, model)

In [61]:
toxicity_results = XGBoost_Classifier('TOXICITY')

[11:38:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [62]:
pickle.dump(toxicity_results[1], open('toxicity_xgb.pickle', 'wb'))

In [63]:
profanity_results = XGBoost_Classifier('PROFANITY')

[11:38:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [64]:
pickle.dump(profanity_results[1], open('profanity_xgb.pickle', 'wb'))

In [65]:
sexually_explicit_results = XGBoost_Classifier('SEXUALLY_EXPLICIT')

[11:39:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [66]:
pickle.dump(sexually_explicit_results[1], open('sexually_explicit_xgb.pickle', 'wb'))